# Data extraction

## Load file

In [153]:
import pandas as pd

# load file to df
df = pd.read_csv('./data/output.csv', dtype='object')
df.sample(5)

,type,date,time,duration,distance,origin,destination,total_earning,base_fare,customer_fare,paid_to_driver,paid_to_uber
2064,Black,"May 21, 2024",11:15 AM,6 min 34 sec,1.91 km,"Av. Brigadeiro Luís Antônio, Jardim Paulista -...","Avenida República do Líbano, Moema - São Paulo...",R$11.07,R$11.07,R$18.20,-R$11.07,R$7.13
1102,Black,"Feb 1, 2024",7:48 AM,19 min 41 sec,6.26 km,"Rua Domingos Paiva, Brás - São Paulo - SP, 030...","R. Duarte de Azevedo, Santana - São Paulo - SP...",R$39.40,R$39.40,R$65.66,-R$39.40,R$26.26
2936,Comfort,"Sep 23, 2024",12:16 PM,24 min 49 sec,5.09 km,"Rua Galvão Bueno, Liberdade - São Paulo - SP, ...","Rua Bela Cintra, Jardim Paulista - São Paulo -...",R$19.87,R$19.87,R$33.12,-R$19.87,R$13.25
3029,Black,"Oct 10, 2024",1:28 PM,9 min 1 sec,1.67 km,"R. Bela Cintra, Consolação - São Paulo - SP, 0...","Av. Higienópolis, Higienópolis - São Paulo - S...",R$11.18,R$11.18,R$18.37,-R$11.18,R$7.19
598,UberX,"Dec 20, 2023",9:56 PM,23 min 28 sec,8.91 km,"Avenida Higienópolis, Consolação - São Paulo -...","Rua Marquês de Valença, Móoca - São Paulo - SP...",R$21.41,R$21.41,R$32.94,-R$21.41,R$11.53


In [25]:
# checking null values
df.isnull().sum()

type               91
date               91
time               91
duration           91
distance           91
origin             91
destination        91
total_earning     126
base_fare         126
customer_fare     266
paid_to_driver    266
paid_to_uber      275
dtype: int64

In [98]:
df1 = df.dropna(ignore_index=True)

In [99]:
df1.head()

,type,date,time,duration,distance,origin,destination,total_earning,base_fare,customer_fare,paid_to_driver,paid_to_uber
0,UberX,"Nov 11, 2023",5:28 PM,11 min 32 sec,4.15 km,"Rua Antônio das Chagas, Santo Amaro - São Paul...","R. Geórgia, Brooklin - São Paulo - SP, 04559-0...",R$15.01,R$11.01,R$19.97,-R$15.01,R$4.96
1,UberX,"Nov 11, 2023",5:20 PM,0 sec,---,"Rua Luís Correia de Melo, Santo Amaro - São Pa...","Rua Luís Correia de Melo, Santo Amaro - São Pa...",R$4.22,R$4.22,R$5.25,-R$4.22,R$1.03
2,UberX,"Nov 11, 2023",4:37 PM,27 min 43 sec,13.42 km,"Rua Helena, Itaim Bibi - São Paulo - SP, 04552...","Rua Quipa, Campo Limpo - São Paulo - SP, 05756...",R$27.09,R$27.09,R$34.95,-R$27.09,R$7.86
3,UberX,"Nov 11, 2023",4:19 PM,16 min 18 sec,4.78 km,"Alameda Gabriel Monteiro da Silva, Jardim Amer...","Rua Helena, Itaim Bibi - São Paulo - SP, 04552...",R$13.78,R$13.78,R$22.96,-R$13.78,R$9.18
4,UberX,"Nov 11, 2023",3:57 PM,5 min 29 sec,1.38 km,"R. Turiassu, Perdizes - São Paulo - SP, 05005-...","Rua Ministro Godói, Perdizes - São Paulo - SP,...",R$6.69,R$6.69,R$10.96,-R$6.69,R$4.27


In [100]:
df1.isnull().sum()

type              0
date              0
time              0
duration          0
distance          0
origin            0
destination       0
total_earning     0
base_fare         0
customer_fare     0
paid_to_driver    0
paid_to_uber      0
dtype: int64

# Transform data

In [101]:
df2 = df1.copy()

## Change data to expected formats

### Format dates and duration

In [102]:
from dateutil import parser

def convert_date(date_str):
    return parser.parse(date_str).strftime('%Y-%m-%d')

def convert_time(time_str):
    time_str = parser.parse(time_str)
    return time_str.strftime('%H:%M:%S')

df2['date'] = df2['date'].apply(convert_date)
df2['time'] = df2['time'].apply(convert_time)
df2['datetime'] = pd.to_datetime(df2['date'] + ' '+ df2['time'])

In [103]:
import pytimeparse
from datetime import timedelta

def parse_duration(duration_str):
    duration = pytimeparse.parse(duration_str) 
    return timedelta(seconds=duration)

df2['duration_dt'] = df2['duration'].apply(parse_duration)

In [104]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3167 entries, 0 to 3166
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   type            3167 non-null   object         
 1   date            3167 non-null   object         
 2   time            3167 non-null   object         
 3   duration        3167 non-null   object         
 4   distance        3167 non-null   object         
 5   origin          3167 non-null   object         
 6   destination     3167 non-null   object         
 7   total_earning   3167 non-null   object         
 8   base_fare       3167 non-null   object         
 9   customer_fare   3167 non-null   object         
 10  paid_to_driver  3167 non-null   object         
 11  paid_to_uber    3167 non-null   object         
 12  datetime        3167 non-null   datetime64[ns] 
 13  duration_dt     3167 non-null   timedelta64[ns]
dtypes: datetime64[ns](1), object(12), timede

### Format distance

In [105]:
def get_distance(dist_str):
    dist_str = dist_str.split(' ')
    if len(dist_str) < 2:
        dist_str = 0
    else:
        dist_str = float(dist_str[0])
    return dist_str

df2['distance_km'] = df2['distance'].apply(get_distance)

In [106]:
df2.head()

,type,date,time,duration,distance,origin,destination,total_earning,base_fare,customer_fare,paid_to_driver,paid_to_uber,datetime,duration_dt,distance_km
0,UberX,2023-11-11,17:28:00,11 min 32 sec,4.15 km,"Rua Antônio das Chagas, Santo Amaro - São Paul...","R. Geórgia, Brooklin - São Paulo - SP, 04559-0...",R$15.01,R$11.01,R$19.97,-R$15.01,R$4.96,2023-11-11 17:28:00,0 days 00:11:32,4.15
1,UberX,2023-11-11,17:20:00,0 sec,---,"Rua Luís Correia de Melo, Santo Amaro - São Pa...","Rua Luís Correia de Melo, Santo Amaro - São Pa...",R$4.22,R$4.22,R$5.25,-R$4.22,R$1.03,2023-11-11 17:20:00,0 days 00:00:00,0.00
2,UberX,2023-11-11,16:37:00,27 min 43 sec,13.42 km,"Rua Helena, Itaim Bibi - São Paulo - SP, 04552...","Rua Quipa, Campo Limpo - São Paulo - SP, 05756...",R$27.09,R$27.09,R$34.95,-R$27.09,R$7.86,2023-11-11 16:37:00,0 days 00:27:43,13.42
3,UberX,2023-11-11,16:19:00,16 min 18 sec,4.78 km,"Alameda Gabriel Monteiro da Silva, Jardim Amer...","Rua Helena, Itaim Bibi - São Paulo - SP, 04552...",R$13.78,R$13.78,R$22.96,-R$13.78,R$9.18,2023-11-11 16:19:00,0 days 00:16:18,4.78
4,UberX,2023-11-11,15:57:00,5 min 29 sec,1.38 km,"R. Turiassu, Perdizes - São Paulo - SP, 05005-...","Rua Ministro Godói, Perdizes - São Paulo - SP,...",R$6.69,R$6.69,R$10.96,-R$6.69,R$4.27,2023-11-11 15:57:00,0 days 00:05:29,1.38


### Get geolocation of origin and destination

In [143]:
import googlemaps

key = './data/places_key.txt'
with open(key, 'r') as f:
    key_str = f.read()
gmaps = googlemaps.Client(key_str)

def get_lat_lng(location_str, gmaps=gmaps):
    location = gmaps.geocode(location_str)
    try:
        location_list = list(location[0]['geometry']['location'].values())
        return location_list
    except:
        return None  

In [ ]:

df2['origin_lat_lng'] = df2['origin'].apply(get_lat_lng)

In [160]:
empty_lists = df2[df2['origin_lat_lng'].apply(lambda x: isinstance(x, list) and len(x) == 0)]

print(empty_lists)

Empty DataFrame
Columns: [type, date, time, duration, distance, origin, destination, total_earning, base_fare, customer_fare, paid_to_driver, paid_to_uber, datetime, duration_dt, distance_km, origin_lat_lng, destination_lat_lng]
Index: []


In [144]:
df2['destination_lat_lng'] = df2['destination'].apply(get_lat_lng)

In [145]:
empty_lists = df2[df2['destination_lat_lng'].apply(lambda x: isinstance(x, list) and len(x) == 0)]

print(empty_lists)

Empty DataFrame
Columns: [type, date, time, duration, distance, origin, destination, total_earning, base_fare, customer_fare, paid_to_driver, paid_to_uber, datetime, duration_dt, distance_km, origin_lat_lng, destination_lat_lng]
Index: []


In [149]:
df2.sample(5).T

,947,679,1215,746,1965
type,Comfort,UberX,Black,Black,Black
date,2024-01-25,2024-01-05,2024-02-29,2024-01-12,2024-06-07
time,00:13:00,10:42:00,21:21:00,18:46:00,07:21:00
duration,6 min 53 sec,20 min 0 sec,5 min 50 sec,35 min 55 sec,15 min 21 sec
distance,2.08 km,9.57 km,2.33 km,9.09 km,5.53 km
origin,"R. Maj. Prado, Indianópolis - São Paulo - SP, ...","Rua Desembargador Vicente Penteado, Pinheiros ...","Rua Teodoro Sampaio, Pinheiros - São Paulo - S...","Av. Albert Einstein, Jardim Leonor - São Paulo...","Rua Doutor Eduardo Amaro, Vila Mariana - São P..."
destination,"Av. Aratãs, Indianópolis - São Paulo - SP, 040...","Rua Indiana, Itaim Bibi - Sao Paulo - SP, 0456...","Rua Harmonia, Sumarezinho - São Paulo - SP, 05...","Al. Lorena, Jardins - São Paulo - SP, 01424-00...","Rua Abelardo Pinto Piolin, República - São Pau..."
total_earning,R$8.71,R$17.98,R$12.13,R$53.96,R$23.57
base_fare,R$8.71,R$17.98,R$12.13,R$52.46,R$23.50
customer_fare,R$14.52,R$29.97,R$19.53,R$89.94,R$36.12


### Format values

In [154]:
columns_to_transform = ['total_earning', 'base_fare', 'customer_fare', 'paid_to_driver', 'paid_to_uber']
df2[columns_to_transform] = df2[columns_to_transform].apply(lambda x: x.str.replace('R\$', '', regex=True).astype(float))

In [157]:
df2.sample(5).T

,3145,709,3102,2140,347
type,Black,Comfort,Black,Comfort,Comfort
date,2024-11-26,2024-01-01,2024-11-18,2024-07-21,2023-12-09
time,19:36:00,19:18:00,11:37:00,16:32:00,19:13:00
duration,17 min 5 sec,19 min 25 sec,31 min 4 sec,20 min 54 sec,19 min 48 sec
distance,3.82 km,7.07 km,12.52 km,13.39 km,8.29 km
origin,"Av. Rebouças, Cerqueira César - São Paulo - SP...","R. Maurício F. Klabin, Vila Mariana - São Paul...","Rua Doutor César, Santana - São Paulo - SP, 02...","R. Alagoas, Higienópolis - São Paulo - SP, 012...","Rua Pedro Osório Filho, Cachoeirinha - São Pau..."
destination,"Pç. Ramos de Azevedo, s/n, São Paulo -, 01037-...","Rua Álvaro Luís Roberto de Assumpção, Campo Be...","R. Cantagalo, Tatuapé - São Paulo - SP, 03319-...","Av. Das Nacoes Unidas, Brooklin Paulista - São...","Rua Torres da Barra, Barra Funda - São Paulo -..."
total_earning,22.89,27.07,45.11,32.44,30.24
base_fare,22.89,20.32,45.11,32.44,25.74
customer_fare,32.55,37.47,71.03,54.06,47.18


In [158]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3167 entries, 0 to 3166
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   type                 3167 non-null   object         
 1   date                 3167 non-null   object         
 2   time                 3167 non-null   object         
 3   duration             3167 non-null   object         
 4   distance             3167 non-null   object         
 5   origin               3167 non-null   object         
 6   destination          3167 non-null   object         
 7   total_earning        3167 non-null   float64        
 8   base_fare            3167 non-null   float64        
 9   customer_fare        3167 non-null   float64        
 10  paid_to_driver       3167 non-null   float64        
 11  paid_to_uber         3167 non-null   float64        
 12  datetime             3167 non-null   datetime64[ns] 
 13  duration_dt       

In [159]:
# send df to csv to save dataframe
df2.to_csv('./artifacts/saved_df.csv')